# **NLP2_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """ You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem.  """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [6]:
print(response1.choices[0].message.content)

Parameters:
Let L1 be the litres of first liquid (Don't know).
Let L2 be the litres of second liquid (Don't know).

Objective Function: 
Maximize Quality(Q) = ln(L1*L2) - ( (L1 + 2*L2)^(1/3) ) 

where ln is the Napierian logarithm

Constraints:
1) Stability Constraint: (L1 - ((L1 + L2)/2))^2 + (L2 - ((L1 + L2)/2))^2 <= 10 
2) Capacity Constraint: L1 + L2 <= 80 

Decision Variables:
The decision variables in this case are L1 and L2 i.e the litres of each component to be used in the process.


### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [8]:
Markdown(response2.choices[0].message.content)

Below is a Python Pyomo code to solve the given problem:

```python
from pyomo.environ import *
from math import log

# Create a model
model = ConcreteModel()

# Declare the variables
model.L1 = Var(domain=NonNegativeReals)
model.L2 = Var(domain=NonNegativeReals)

# Define the objective function
def obj_expression(model): 
    return log(model.L1*model.L2 - (model.L1 + 2*model.L2)**(1/3))
model.OBJ = Objective(rule=obj_expression, sense=maximize)

# Define the constraints
def stability_const(model):
    return (model.L1 - (model.L1 + model.L2)/2)**2 + (model.L2 - (model.L1 + model.L2)/2)**2 <= 10
model.Constraint1 = Constraint(rule=stability_const)

def capacity_const(model):
    return model.L1 + model.L2 <= 80
model.Constraint2 = Constraint(rule=capacity_const)

# Solve the problem
solver = SolverFactory('glpk')
solver.solve(model)
```

Please note that this code may not give intended results because optimization of logarithms is challenging due to its non-convex and discontinuous nature. It would be good to apply logarithm transformation to the objective and constraints or use another corresponding mathematical trick.

This code uses the 'glpk' solver. Make sure it is installed in your Python environment. If not, you can substitute 'glpk' with other available solver in your environment, like 'gurobi', 'cplex', etc. Or, you can install the GLPK solver using the package manager suitable for your Python environment (like pip, conda).


### **Run the code Generated by GPT4**

In [11]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.9 MB/s eta 0:00:00


In [12]:

_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [15]:
from pyomo.environ import *
from math import log

# Create a model
model = ConcreteModel()

# Declare the variables
model.L1 = Var(domain=NonNegativeReals)
model.L2 = Var(domain=NonNegativeReals)

# Define the objective function
def obj_expression(model):
    return log(model.L1*model.L2 - (model.L1 + 2*model.L2)**(1/3))
model.OBJ = Objective(rule=obj_expression, sense=maximize)

# Define the constraints
def stability_const(model):
    return (model.L1 - (model.L1 + model.L2)/2)**2 + (model.L2 - (model.L1 + model.L2)/2)**2 <= 10
model.Constraint1 = Constraint(rule=stability_const)

def capacity_const(model):
    return model.L1 + model.L2 <= 80
model.Constraint2 = Constraint(rule=capacity_const)

# Solve the problem
solver = SolverFactory('knitro')
solver.solve(model)

ERROR:pyomo.core:Rule failed when generating expression for Objective OBJ with index None:
TypeError: Implicit conversion of Pyomo numeric value (L1*L2 - (L1 + 2*L2)**0.3333333333333333) to float is disabled.
This error is often the result of using Pyomo components as arguments to
one of the Python built-in math module functions when defining
expressions. Avoid this error by using Pyomo-provided math functions or
explicitly resolving the numeric value using the Pyomo value() function.
ERROR:pyomo.core:Constructing component 'OBJ' from data=None failed:
    TypeError: Implicit conversion of Pyomo numeric value (L1*L2 - (L1 + 2*L2)**0.3333333333333333) to float is disabled.
This error is often the result of using Pyomo components as arguments to
one of the Python built-in math module functions when defining
expressions. Avoid this error by using Pyomo-provided math functions or
explicitly resolving the numeric value using the Pyomo value() function.


TypeError: Implicit conversion of Pyomo numeric value (L1*L2 - (L1 + 2*L2)**0.3333333333333333) to float is disabled.
This error is often the result of using Pyomo components as arguments to
one of the Python built-in math module functions when defining
expressions. Avoid this error by using Pyomo-provided math functions or
explicitly resolving the numeric value using the Pyomo value() function.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [20]:
from pyomo.environ import *
from pyomo.core.expr.current import log

# Create a model
model = ConcreteModel()

# Declare the variables
model.L1 = Var(domain=NonNegativeReals)
model.L2 = Var(domain=NonNegativeReals)

# Define the objective function
def obj_expression(model):
    return log(model.L1*model.L2) - (model.L1 + 2*model.L2)**(1/3)
model.OBJ = Objective(rule=obj_expression, sense=maximize)

# Define the constraints
def stability_const(model):
    return (model.L1 - (model.L1 + model.L2)/2)**2 + (model.L2 - (model.L1 + model.L2)/2)**2 <= 10
model.Constraint1 = Constraint(rule=stability_const)

def capacity_const(model):
    return model.L1 + model.L2 <= 80
model.Constraint2 = Constraint(rule=capacity_const)

# Solve the problem
solver = SolverFactory('knitro')
solver.solve(model)
print(model.L1())
print(model.L2())

42.23606797533548
37.76393202225084
